In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from spacy.cli.train import train
import spacy


2023-04-19 16:13:14.026789: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [26]:
from transformers import AutoTokenizer, AutoModel
_ = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
_ = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [18]:
train("config/config_bert.cfg",
      output_path='sig_ner_bert2_model',
      overrides={"paths.train": "train_docs.spacy", 
                 "paths.dev": "test_docs.spacy",
                 "training.max_epochs": 10})

!python3 -m spacy train config/config_bert.cfg -o sig_bert_model --paths.train train_docs.spacy --paths.dev test_docs.spacy --training.max_epochs 8


✔ Created output directory: sig_ner_bert2_model
ℹ Saving to output directory: sig_ner_bert2_model
ℹ Using CPU
ℹ To switch to GPU 0, use the option: --gpu-id 0

=========================== Initializing pipeline ===========================


Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['transformer', 'ner']
ℹ Initial learn rate: 0.0
E    #       LOSS TRANS...  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  -------------  --------  ------  ------  ------  ------
  0       0       29214.57   1228.53    0.09    0.11    0.08    0.00
  3     200      595007.49  86022.72   63.14   76.26   53.87    0.63
  6     400       53470.24  34592.91   86.61   84.83   88.48    0.87
✔ Saved pipeline to output directory
sig_ner_bert2_model/model-last
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
2023-04-20 09:51:19.067891: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow bin

In [23]:
trained_nlp = spacy.load('./sig_bert_model/model-best')

##### first outcome! 

In [24]:
inp = 'TAKE 1 TABLET  (150 MCG TOTAL) BY MOUTH two times DAILY for one week by the way patient id is 303023 and take 2 aderol every day'.lower()
[(e, e.label_) for e in trained_nlp(inp).ents]

[(1, 'Dosage'),
 (tablet, 'Form'),
 (150 mcg, 'Strength'),
 (two times daily, 'Frequency'),
 (for one week, 'Duration'),
 (2, 'Dosage'),
 (aderol, 'Drug'),
 (every day, 'Frequency')]

In [17]:
inp = 'Take 2 tablets of aderol in the morning every week'.lower()
[(e, e.label_) for e in trained_nlp(inp).ents]

[(2, 'Dosage'), (tablets, 'Form'), (aderol, 'Drug'), (every week, 'Frequency')]

In [31]:
inp = "Take 2 capsules qid of amoxicillin 500mg"
[(e, e.label_) for e in trained_nlp(inp).ents]

[(2, 'Dosage'),
 (capsules, 'Form'),
 (qid, 'Frequency'),
 (amoxicillin, 'Drug'),
 (500mg, 'Strength')]

 We can even parse multuple dosing instructions

In [25]:
inp = 'inhale 3 puffs of albuterol 2 times a day for one week' \
      ' then 1 puff of albuterol every day for 2 months'

print([(e, e.label_) for e in trained_nlp(inp).ents])

[(3 puffs, 'Dosage'), (albuterol, 'Drug'), (2 times a day, 'Frequency'), (for one week, 'Duration'), (1 puff, 'Dosage'), (albuterol, 'Drug'), (every day, 'Frequency'), (for 2 months, 'Duration')]


In [20]:
inp = 'TAKE 1 TABLET (20 MG) BY MOUTH every 2 months'.lower()
print([(e, e.label_) for e in trained_nlp(inp).ents])

[(tablet, 'Form'), (20 mg, 'Strength')]


 There are many things to improve, one example is the frequency that can and should be parsed to an interval type and amoubt (e.g type=day, amount=2)

Also, the model only recognizes encountered features, so inputting to the model a new sentence, will not be parsed correctly. 
When we will use the pre-trained Bert model, this issue should be resolved

In [11]:
inp = 'Take 1 tablet of ibuprofen 3 times a day'


[(e, e.label_) for e in trained_nlp(inp).ents]

[(1, 'Dosage'),
 (tablet, 'Form'),
 (ibuprofen, 'Drug'),
 (3 times a day, 'Dosage')]

In [51]:
inp = '1 TAB Q7d'.lower()

[(e, e.label_) for e in trained_nlp(inp).ents]

[(1, 'Dosage'), (q7d, 'Frequency')]